In [ ]:
import glob
import random
import numpy as np
from written_test_automation import pre, loc, result, nets, config
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def s(im):
    plt.imshow(im,cmap="gray")
    plt.show()

# Net trainer

In [ ]:
weight_path = "./written_test_automation/answer_weights.pickle"
append_path = "./answer_images/"
folders_label =[(append_path + "0/*",[1,0,0,0,0,0]),
                (append_path + "1/*",[0,1,0,0,0,0]),
                (append_path + "2/*",[0,0,1,0,0,0]),
                (append_path + "3/*",[0,0,0,1,0,0]),
                (append_path + "4/*",[0,0,0,0,1,0]),
                (append_path + "5/*",[0,0,0,0,0,1])]


In [ ]:
paths = []
labels = []
for path,label in folders_label:
    new_paths = glob.glob(path)
    labels = labels + ([label] * len(new_paths))
    paths = paths + new_paths
paths,p, labels,l = train_test_split(paths, labels, test_size=0.01)
print(p)
print(l)

In [ ]:
x = []
for path in paths:
    x.append([pre.imread(path=path)])
    
x = np.array(x, dtype = np.float64 ) / 255
y = np.array(labels, dtype = np.float64)

In [ ]:
print(x.shape)
print(x.min())
print(x.max())

In [ ]:
net = nets.answer_model()
net.load_weights(weight_path)

In [ ]:
max_size = int(x.shape[0])
batch_size = 50
epochs = 25

for _ in range(epochs):
    print(_)
    start = 0
    fin = start + batch_size
    while True:
        if (start > max_size) or (fin > max_size):
            break
        net.numpy_train(x[start:fin],y[start:fin], epochs=1)
        start += batch_size
        fin += batch_size

In [ ]:
net.dropout = nets.nn.Dropout(0.0)
net.numpy_train(x, y, epochs=3)

In [ ]:
net.save_weights(weight_path)

# Config finder

In [ ]:

def pick(ims, to_print):
    '''
        threshold_kernel, blur_kernel, blob_size
    '''
    th_ker = random.uniform(0.00001,0.1)
    blur_kernel = random.uniform(0.00001,0.1)
    blob_size = 0.037#random.uniform(0.00001,0.1)
    C = 7 #random.randint(2,7)
    
    config.threshold_kernel_percent = th_ker
    config.median_blur_kernel_percent = blur_kernel
    config.blob_min_size_percent = blob_size
    config.threshold_C = C
    print(to_print ,th_ker, blur_kernel, blob_size, C)


    try:
        for im in ims:
            result.predict(im)
        print("yes")
        return [th_ker,blur_kernel,blob_size, C]
    except:
        return None
def dir_pick(path, loops = 100):
    paths = glob.glob(path)
    ims = []
    for i in paths:
        print(i)
        ims.append(pre.imread(i))

        
    configs = []
    for i in range(loops):
        picked_config = pick(ims, str(i))
        
        if picked_config == None:
            pass
        else:
            configs = configs + [picked_config]
            
    return configs

In [ ]:
con = dir_pick("./im/*", 100)

In [ ]:
print(np.mean(con, axis=0))
print(np.median(con, axis=0))

In [ ]:
con